In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [65]:
# Import some of the most important libary for using this notebook
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [66]:
file_path = 'Data.csv'
# Read csv file for using pandas libary
sarcasm_df = pd.read_csv(file_path)
# Looked first five rows

label_mapping = {'Sarcastic': 1, 'Non Sarcastic': 0}
sarcasm_df['target'] = sarcasm_df['target'].map(label_mapping)
sarcasm_df.head()

,headlines,target
0,CNN Triumphs (At Least in Most Demographic Cat...,0
1,"‘You Did The Best You Could,’ Says Iron Man Ac...",1
2,New Emails Reveal Warm Relationship Between Ka...,0
3,Donald Trump Jr. Gets Slammed Over Racist Birt...,0
4,God Urges Rick Perry Not To Run For President,1


In [67]:
vocab_size = 10000 # It's most common vocabulary size.
embedding_dim = 16 
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [115]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sarcasm_df['headlines'])
sequences = tokenizer.texts_to_sequences(sarcasm_df['headlines'])

# Pad sequences to ensure uniform input length
X = pad_sequences(sequences, maxlen=max_length)
y = sarcasm_df['target'].values

In [117]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,
                                                 random_state=42)

len(x_train),len(x_test),len(y_train),len(y_test)

(10004, 2502, 10004, 2502)

In [118]:
# Create a Callback for using when I train model.
e_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    restore_best_weights=True,
)
bm_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False  # If you want to save the full model, not just the weights
)


In [119]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

Epoch 1/5


c:\Users\AS COMPUTER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 65ms/step - accuracy: 0.7707 - loss: 0.4388 - val_accuracy: 0.9049 - val_loss: 0.2437
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.9602 - loss: 0.1129 - val_accuracy: 0.9053 - val_loss: 0.2342
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.9857 - loss: 0.0516 - val_accuracy: 0.9009 - val_loss: 0.3184
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.9944 - loss: 0.0219 - val_accuracy: 0.8957 - val_loss: 0.4016
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.9955 - loss: 0.0180 - val_accuracy: 0.8989 - val_loss: 0.4686


In [120]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,087,493 (15.59 MB)

 Trainable params: 1,362,497 (5.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,724,996 (10.40 MB)

In [121]:
model.save('sarcasm_detect.h5')

In [122]:
import tensorflow as tf
load_model  = tf.keras.models.load_model('sarcasm_detect.h5')
load_model.evaluate(x_test,tf.expand_dims(y_test,axis=1))

79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9124 - loss: 0.4137


[0.46856170892715454, 0.8988808989524841]

In [127]:
for i, (test_input_seq, actual_label) in enumerate(zip(x_test[50:], y_test[50:])):
    
    # Ensure the input sequence is in the right format
    test_input_seq = pad_sequences([test_input_seq], maxlen=max_length, padding=padding_type, truncating=trunc_type)
    
    # Predict sarcasm
    probs = load_model.predict(test_input_seq)
    pred = (probs > 0.5).astype(int)[0][0]  # Interpret the probability output
    
    # Print results
    print(f"Prediction Value Is: {pred}")
    print(f'Actual Value Is: {actual_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Pr

KeyboardInterrupt: 